In [ ]:
import os
import time
from datetime import datetime
import requests
from pymongo import MongoClient


class MaoYan(object):
    """
    猫眼评论爬虫，爬取电影《流浪地球》的评论和评分
    """

    def __init__(self):
        """
        初始化函数
        :param
        headers: 请求头
        time: 当前时间戳
        premiere_time: 首映时间的时间戳
        """
        self.headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.'
                                      '38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
                        'Connection': 'keep-alive',
                        'Cookie': '_lxsdk_cuid=168d5d128e7c8-033114908a580c-10376654-fa000-168d5d128e7c8;'
                                  ' _lx_utm=utm_source%3Dbing%26utm_medium%3Dorganic; uuid_n_v=v1;'
                                  ' iuuid=5D49FF702DB211E9AF1B8D0648275EC02D381B7848144BC1A299A63C05094BF5;'
                                  ' webp=true; selectci=true; ci=281%2C%E6%83%A0%E5%B7%9E;'
                                  ' __mta=247299643.1549775481575.1549783540088.1549862773375.3;'
                                  ' _lxsdk=5D49FF702DB211E9AF1B8D0648275EC02D381B7848144BC1A299A63C05094BF5;'
                                  ' _lxsdk_s=168db05185a-332-e0d-bc5%7C%7C157'}
        self.time = int(time.time()*1000)
        self.premiere_time = int(time.mktime(time.strptime('2019-02-05 00:00:00', '%Y-%m-%d %H:%M:%S'))*1000)

        # 配置mongodb数据库
        host = os.environ.get('MONGODB_HOST', '127.0.0.1')  # 本地数据库
        port = os.environ.get('MONGODB_PORT', '27017')  # 数据库端口
        mongo_url = 'mongodb://{}:{}'.format(host, port)
        mongo_db = os.environ.get('MONGODB_DATABASE', 'maoyan')
        client = MongoClient(mongo_url)
        self.db = client[mongo_db]
        self.db['maoyan'].create_index('id', unique=True)  # 以评论的id为主键进行去重

    def get_comment(self):
        """
        爬取首映到当前时间的电影评论
        :param
        url: 评论真实请求的url，参数ts为时间戳
        :return: None
        """
        url = 'http://m.maoyan.com/review/v2/comments.json?movieId=248906&userId=-1&' \
              'offset=0&limit=15&ts={}&type=3'
        while self.time > self.premiere_time:
            req_url = url.format(self.time)
            res = requests.get(req_url, headers=self.headers)
            #time.sleep(1)
            count = 0
            for com in res.json()['data']['comments']:
                self.parse_comment(com=com)
                count += 1
                if count == 15:
                    self.time = com['time']

            print('成功爬取截止到{}的数据！'.format(datetime.fromtimestamp(int(self.time/1000))))

    def parse_comment(self, com):
        """
        解析函数，用来解析爬回来的json评论数据，并把数据保存进mongodb数据库
        :param com: 每一条评论的json数据
        :return:
        """
        comment = {'content': com['content'], 'gender': com['gender'], 'id': com['id'],
                   'nick': com['nick'], 'replyCount': com['replyCount'], 'score': com['score'],
                   'time': com['time'], 'upCount': com['upCount'],
                   'userId': com['userId'], 'userLevel': com['userLevel']}  # 构造评论字典
        # 通过评论id去重，如果已经有了就更新，没有就插入
        self.db['maoyan'].update_one({'id': comment['id']}, {'$set': comment}, upsert=True)


if __name__ == '__main__':
    my = MaoYan()
    my.get_comment()
    

成功爬取截止到2019-02-17 08:30:00的数据！
成功爬取截止到2019-02-17 08:23:00的数据！
成功爬取截止到2019-02-17 08:18:00的数据！
成功爬取截止到2019-02-17 08:12:00的数据！
成功爬取截止到2019-02-17 08:04:00的数据！
成功爬取截止到2019-02-17 07:48:00的数据！
成功爬取截止到2019-02-17 07:40:00的数据！
成功爬取截止到2019-02-17 07:22:00的数据！
成功爬取截止到2019-02-17 06:57:00的数据！
成功爬取截止到2019-02-17 06:38:00的数据！
成功爬取截止到2019-02-17 05:54:00的数据！
成功爬取截止到2019-02-17 04:18:00的数据！
成功爬取截止到2019-02-17 03:11:00的数据！
成功爬取截止到2019-02-17 02:46:00的数据！
成功爬取截止到2019-02-17 02:38:00的数据！
成功爬取截止到2019-02-17 02:26:00的数据！
成功爬取截止到2019-02-17 02:16:00的数据！
成功爬取截止到2019-02-17 02:06:00的数据！
成功爬取截止到2019-02-17 01:58:00的数据！
成功爬取截止到2019-02-17 01:51:00的数据！
成功爬取截止到2019-02-17 01:42:00的数据！
成功爬取截止到2019-02-17 01:33:00的数据！
成功爬取截止到2019-02-17 01:29:00的数据！
成功爬取截止到2019-02-17 01:25:00的数据！
成功爬取截止到2019-02-17 01:21:00的数据！
成功爬取截止到2019-02-17 01:17:00的数据！
成功爬取截止到2019-02-17 01:14:00的数据！
成功爬取截止到2019-02-17 01:11:00的数据！
成功爬取截止到2019-02-17 01:09:00的数据！
成功爬取截止到2019-02-17 01:05:00的数据！
成功爬取截止到2019-02-17 01:02:00的数据！
成功爬取截止到2019-02-17 00:59:00的数据！
成功爬取截止到2

成功爬取截止到2019-02-16 18:34:00的数据！
成功爬取截止到2019-02-16 18:32:00的数据！
成功爬取截止到2019-02-16 18:30:00的数据！
成功爬取截止到2019-02-16 18:29:00的数据！
成功爬取截止到2019-02-16 18:28:00的数据！
成功爬取截止到2019-02-16 18:25:00的数据！
成功爬取截止到2019-02-16 18:23:00的数据！
成功爬取截止到2019-02-16 18:22:00的数据！
成功爬取截止到2019-02-16 18:20:00的数据！
成功爬取截止到2019-02-16 18:19:00的数据！
成功爬取截止到2019-02-16 18:17:00的数据！
成功爬取截止到2019-02-16 18:15:00的数据！
成功爬取截止到2019-02-16 18:13:00的数据！
成功爬取截止到2019-02-16 18:11:00的数据！
成功爬取截止到2019-02-16 18:10:00的数据！
成功爬取截止到2019-02-16 18:09:00的数据！
成功爬取截止到2019-02-16 18:08:00的数据！
成功爬取截止到2019-02-16 18:06:00的数据！
成功爬取截止到2019-02-16 18:04:00的数据！
成功爬取截止到2019-02-16 18:02:00的数据！
成功爬取截止到2019-02-16 18:00:00的数据！
成功爬取截止到2019-02-16 17:59:00的数据！
成功爬取截止到2019-02-16 17:58:00的数据！
成功爬取截止到2019-02-16 17:57:00的数据！
成功爬取截止到2019-02-16 17:55:00的数据！
成功爬取截止到2019-02-16 17:53:00的数据！
成功爬取截止到2019-02-16 17:51:00的数据！
成功爬取截止到2019-02-16 17:50:00的数据！
成功爬取截止到2019-02-16 17:49:00的数据！
成功爬取截止到2019-02-16 17:47:00的数据！
成功爬取截止到2019-02-16 17:45:00的数据！
成功爬取截止到2019-02-16 17:44:00的数据！
成功爬取截止到2

成功爬取截止到2019-02-16 00:36:00的数据！
成功爬取截止到2019-02-16 00:34:00的数据！
成功爬取截止到2019-02-16 00:32:00的数据！
成功爬取截止到2019-02-16 00:30:00的数据！
成功爬取截止到2019-02-16 00:27:00的数据！
成功爬取截止到2019-02-16 00:25:00的数据！
成功爬取截止到2019-02-16 00:23:00的数据！
成功爬取截止到2019-02-16 00:21:00的数据！
成功爬取截止到2019-02-16 00:19:00的数据！
成功爬取截止到2019-02-16 00:18:00的数据！
成功爬取截止到2019-02-16 00:16:00的数据！
成功爬取截止到2019-02-16 00:14:00的数据！
成功爬取截止到2019-02-16 00:12:00的数据！
成功爬取截止到2019-02-16 00:10:00的数据！
成功爬取截止到2019-02-16 00:09:00的数据！
成功爬取截止到2019-02-16 00:08:00的数据！
成功爬取截止到2019-02-16 00:06:00的数据！
成功爬取截止到2019-02-16 00:04:00的数据！
成功爬取截止到2019-02-16 00:02:00的数据！
成功爬取截止到2019-02-16 00:00:00的数据！
成功爬取截止到2019-02-15 23:59:00的数据！
成功爬取截止到2019-02-15 23:57:00的数据！
成功爬取截止到2019-02-15 23:55:00的数据！
成功爬取截止到2019-02-15 23:53:00的数据！
成功爬取截止到2019-02-15 23:51:00的数据！
成功爬取截止到2019-02-15 23:49:00的数据！
成功爬取截止到2019-02-15 23:48:00的数据！
成功爬取截止到2019-02-15 23:46:00的数据！
成功爬取截止到2019-02-15 23:44:00的数据！
成功爬取截止到2019-02-15 23:42:00的数据！
成功爬取截止到2019-02-15 23:40:00的数据！
成功爬取截止到2019-02-15 23:39:00的数据！
成功爬取截止到2

成功爬取截止到2019-02-15 17:16:00的数据！
成功爬取截止到2019-02-15 17:14:00的数据！
成功爬取截止到2019-02-15 17:12:00的数据！
成功爬取截止到2019-02-15 17:11:00的数据！
成功爬取截止到2019-02-15 17:09:00的数据！
成功爬取截止到2019-02-15 17:07:00的数据！
成功爬取截止到2019-02-15 17:05:00的数据！
成功爬取截止到2019-02-15 17:03:00的数据！
成功爬取截止到2019-02-15 17:01:00的数据！
成功爬取截止到2019-02-15 16:59:00的数据！
成功爬取截止到2019-02-15 16:57:00的数据！
成功爬取截止到2019-02-15 16:55:00的数据！
成功爬取截止到2019-02-15 16:53:00的数据！
成功爬取截止到2019-02-15 16:51:00的数据！
成功爬取截止到2019-02-15 16:49:00的数据！
成功爬取截止到2019-02-15 16:48:00的数据！
成功爬取截止到2019-02-15 16:46:00的数据！
成功爬取截止到2019-02-15 16:44:00的数据！
成功爬取截止到2019-02-15 16:42:00的数据！
成功爬取截止到2019-02-15 16:40:00的数据！
成功爬取截止到2019-02-15 16:39:00的数据！
成功爬取截止到2019-02-15 16:38:00的数据！
成功爬取截止到2019-02-15 16:36:00的数据！
成功爬取截止到2019-02-15 16:35:00的数据！
成功爬取截止到2019-02-15 16:33:00的数据！
成功爬取截止到2019-02-15 16:31:00的数据！
成功爬取截止到2019-02-15 16:29:00的数据！
成功爬取截止到2019-02-15 16:27:00的数据！
成功爬取截止到2019-02-15 16:24:00的数据！
成功爬取截止到2019-02-15 16:22:00的数据！
成功爬取截止到2019-02-15 16:19:00的数据！
成功爬取截止到2019-02-15 16:16:00的数据！
成功爬取截止到2

成功爬取截止到2019-02-15 00:47:00的数据！
成功爬取截止到2019-02-15 00:45:00的数据！
成功爬取截止到2019-02-15 00:43:00的数据！
成功爬取截止到2019-02-15 00:42:00的数据！
成功爬取截止到2019-02-15 00:41:00的数据！
成功爬取截止到2019-02-15 00:40:00的数据！
成功爬取截止到2019-02-15 00:39:00的数据！
成功爬取截止到2019-02-15 00:38:00的数据！
成功爬取截止到2019-02-15 00:37:00的数据！
成功爬取截止到2019-02-15 00:35:00的数据！
成功爬取截止到2019-02-15 00:34:00的数据！
成功爬取截止到2019-02-15 00:32:00的数据！
成功爬取截止到2019-02-15 00:31:00的数据！
成功爬取截止到2019-02-15 00:29:00的数据！
成功爬取截止到2019-02-15 00:27:00的数据！
成功爬取截止到2019-02-15 00:25:00的数据！
成功爬取截止到2019-02-15 00:24:00的数据！
成功爬取截止到2019-02-15 00:23:00的数据！
成功爬取截止到2019-02-15 00:22:00的数据！
成功爬取截止到2019-02-15 00:21:00的数据！
成功爬取截止到2019-02-15 00:20:00的数据！
成功爬取截止到2019-02-15 00:19:00的数据！
成功爬取截止到2019-02-15 00:18:00的数据！
成功爬取截止到2019-02-15 00:16:00的数据！
成功爬取截止到2019-02-15 00:15:00的数据！
成功爬取截止到2019-02-15 00:14:00的数据！
成功爬取截止到2019-02-15 00:13:00的数据！
成功爬取截止到2019-02-15 00:12:00的数据！
成功爬取截止到2019-02-15 00:11:00的数据！
成功爬取截止到2019-02-15 00:10:00的数据！
成功爬取截止到2019-02-15 00:09:00的数据！
成功爬取截止到2019-02-15 00:08:00的数据！
成功爬取截止到2

成功爬取截止到2019-02-14 19:39:00的数据！
成功爬取截止到2019-02-14 19:37:00的数据！
成功爬取截止到2019-02-14 19:35:00的数据！
成功爬取截止到2019-02-14 19:34:00的数据！
成功爬取截止到2019-02-14 19:33:00的数据！
成功爬取截止到2019-02-14 19:31:00的数据！
成功爬取截止到2019-02-14 19:29:00的数据！
成功爬取截止到2019-02-14 19:27:00的数据！
成功爬取截止到2019-02-14 19:26:00的数据！
成功爬取截止到2019-02-14 19:25:00的数据！
成功爬取截止到2019-02-14 19:24:00的数据！
成功爬取截止到2019-02-14 19:23:00的数据！
成功爬取截止到2019-02-14 19:21:00的数据！
成功爬取截止到2019-02-14 19:19:00的数据！
成功爬取截止到2019-02-14 19:17:00的数据！
成功爬取截止到2019-02-14 19:15:00的数据！
成功爬取截止到2019-02-14 19:13:00的数据！
成功爬取截止到2019-02-14 19:12:00的数据！
成功爬取截止到2019-02-14 19:10:00的数据！
成功爬取截止到2019-02-14 19:09:00的数据！
成功爬取截止到2019-02-14 19:07:00的数据！
成功爬取截止到2019-02-14 19:06:00的数据！
成功爬取截止到2019-02-14 19:04:00的数据！
成功爬取截止到2019-02-14 19:03:00的数据！
成功爬取截止到2019-02-14 19:01:00的数据！
成功爬取截止到2019-02-14 18:59:00的数据！
成功爬取截止到2019-02-14 18:58:00的数据！
成功爬取截止到2019-02-14 18:56:00的数据！
成功爬取截止到2019-02-14 18:54:00的数据！
成功爬取截止到2019-02-14 18:53:00的数据！
成功爬取截止到2019-02-14 18:51:00的数据！
成功爬取截止到2019-02-14 18:49:00的数据！
成功爬取截止到2